Print path of current directory:

In [ ]:
;pwd

Activate `Apophis.jl` environment:

In [ ]:
]activate ..

In [ ]:
]st

In [ ]:
#using Revise

In [ ]:
using Apophis

In [ ]:
using PlanetaryEphemeris #: yr, au, c_au_per_day # these variables are already exported

In [ ]:
using TaylorIntegration

In [ ]:
methods(TaylorIntegration.jetcoeffs!)

In [ ]:
using Plots, DelimitedFiles, Dates
using Statistics: mean, std
using AstroTime
using Printf

Load Sun, 8 planets, Moon, Pluto and 16 asteroid ephemeris, previously integrated with TaylorIntegration.jl

In [ ]:
using JLD
using Statistics: mean, std

In [ ]:
#Load time (t) and state (x) arrays, to construct a TaylorInterpolant object
#ss16ast_eph_t = load(joinpath(Apophis.jplephpath, "ss16ast343_eph_minus5yr_tx_BACKWARDS.jld"), "ss16ast_eph_t");
#ss16ast_eph_x = load(joinpath(Apophis.jplephpath, "ss16ast343_eph_minus5yr_tx_BACKWARDS.jld"), "ss16ast_eph_x");
#ss16ast_eph_t = load(joinpath(Apophis.jplephpath, "ss16ast343_eph_24yr_tx.jld"), "ss16ast_eph_t");
#ss16ast_eph_x = load(joinpath(Apophis.jplephpath, "ss16ast343_eph_24yr_tx.jld"), "ss16ast_eph_x");
ss16ast_eph_t = load(joinpath(Apophis.jplephpath, "ss16ast343_eph_5yr_tx.jld"), "ss16ast_eph_t");
ss16ast_eph_x = load(joinpath(Apophis.jplephpath, "ss16ast343_eph_5yr_tx.jld"), "ss16ast_eph_x");

In [ ]:
length(ss16ast_eph_t)

In [ ]:
mean(diff(ss16ast_eph_t))

In [ ]:
std(diff(ss16ast_eph_t))

In [ ]:
ss16asteph = TaylorInterpolant(ss16ast_eph_t, ss16ast_eph_x);

In [ ]:
ss16asteph(2454733.5)

In [ ]:
ss16asteph(2454733.5+5*365.25)

In [ ]:
ss16asteph(2454733.5-5*365.25)

In [ ]:
ss16asteph(2454733.5+14*365.25)

In [ ]:
ss16asteph(2454733.5+24*365.25)

Load Apophis ephemeris (also integrated previously with TaylorIntegration.jl); dynamical model includes: post-Newtonian interactions with Sun, 8 planets, Moon, Pluto, and 16 asteroids (same as those included in JPL's solution #199 for Apophis). It also includes interaction with Earth's J2 and J3. Earth's orientation model is the same as DE430/431.

In [ ]:
#recover integration from .jld file
vars = ["t", "x"] #names of variables
filename = "Apophis_jt1_p5y_ic199.jld"
#filename = "Apophis_jt_24yr_tx_tSS.jld"
#filename = "Apophis_jt_24yr_tx.jld"
#filename = "Apophis_jt_5yr_tx.jld"
for i in eachindex(vars)
    ex = Symbol(vars[i])
    @eval $ex = load($filename, vars[$i])
end

In [ ]:
apophis = TaylorInterpolant(t, x)
@show apophis.t[1]
const t0 = apophis.t[1]
#tv1 = apophis.t

In [ ]:
julian2datetime(apophis.t[end])

In [ ]:
apophis_et(et) = apophis( etsecs2julian(et) )[1:6]
earth_et(et) = ss16asteph( etsecs2julian(et) )[union(3*4-2:3*4,3*(27+4)-2:3*(27+4))]
sun_et(et) = ss16asteph( etsecs2julian(et) )[union(3*1-2:3*1,3*(27+1)-2:3*(27+1))]

In [ ]:
apophis(t0+4yr)

In [ ]:
apophis(t0+5yr)

In [ ]:
apophis(t0+24yr)

Load Apophis radar astrometry, downloaded from JPL's radar astrometry website: https://ssd.jpl.nasa.gov/?radar

In [ ]:
apophis_radar_data_2005_2013 = process_radar_data_jpl("../Apophis_JPL_data.dat")

Construct vector of observation times in UTC and TDB:

In [ ]:
#construct vector of observation times (UTC)
tv_jpl_utc = UTCEpoch.([x.utcepoch for x in apophis_radar_data_2005_2013])
# convert to TDB
tv_jpl_tdb = TDBEpoch.(tv_jpl_utc)
# date/time to Julian date
tv_jpl_tdb_julian =  map(x->x.Δt, julian.(tv_jpl_tdb))
# construct time range variable with t0 and observation times > t0, removing repeated values
tv = union(t0, tv_jpl_tdb_julian[tv_jpl_tdb_julian .> t0])
@show all(diff(tv) .> 0)

Take only observations from 2012-2013:

In [ ]:
apophis_data = apophis_radar_data_2005_2013[6:end] #take only the 2012-2013 subset

Check that each receiver and transmitter antennas are the same (i.e., monostatic mode):

In [ ]:
@show all( ismonostatic.(apophis_data) )

Get the station (observatories) codes:

In [ ]:
#then, get the station codes
station_codes = [x.rcvr for x in apophis_data];

Get transmission frequencies (MHz):

In [ ]:
#extract transmitter frequencies (MHz)
transmitter_freq_MHz = [x.freq for x in apophis_data]

Get UTC receive times:

In [ ]:
# get UTC receive times
tv_jpl_utc_jul = [datetime2julian(x.utcepoch) for x in apophis_data];

Get indices of delay and Doppler observations in `apophis_data`:

In [ ]:
delay_index = findall(x->x.delay_units=="us", apophis_data)
doppler_index = findall(x->x.doppler_units=="Hz", apophis_data);

# Delay/Doppler observations reduction

In [ ]:
niters = 3 # number of iterations in time delay recursive process
rad_ind = 2

In [ ]:
station_codes[rad_ind]

In [ ]:
UTCEpoch.(TDBEpoch.(tv, origin=:julian))

In [ ]:
apophis_data[rad_ind].utcepoch

In [ ]:
apophis_data[rad_ind].freq

In [ ]:
Apophis.loadjpleph()

In [ ]:
tdelay, dshift = delay_doppler(
    apophis_data[rad_ind].rcvr,
    apophis_data[rad_ind].utcepoch,
    apophis_data[rad_ind].freq,
    niters,
    xve = earth_et,
    xvs = sun_et,
    xva = apophis_et
)

In [ ]:
tdelay(), dshift()

In [ ]:
tdelay(-5.762), dshift(-5.762)

In [ ]:
tdelay_jpl, dshift_jpl = delay_doppler(
    apophis_data[rad_ind].rcvr,
    apophis_data[rad_ind].utcepoch,
    apophis_data[rad_ind].freq,
    niters#,
    #xva=Apophis.apophis_pv_199
)

In [ ]:
apophis_data[rad_ind].delay #DELAY observed value

In [ ]:
tdelay_jpl #DELAY computed value, JPL

In [ ]:
apophis_data[rad_ind].delay - tdelay_jpl #DELAY O-C, JPL

In [ ]:
tdelay() #DELAY computed value, TaylorIntegration.jl, with A2=0

In [ ]:
apophis_data[rad_ind].delay - tdelay() #DELAY O-C, TaylorIntegration, with A2=0

In [ ]:
apophis_data[rad_ind].delay - tdelay(-5.772) #DELAY O-C, TaylorIntegration, with A2=-5.7

In [ ]:
apophis_data[rad_ind].doppler #DOPPLER observed value

In [ ]:
dshift_jpl #DOPPLER computed value, JPL

In [ ]:
apophis_data[rad_ind].doppler - dshift_jpl #DOPPLER O-C, JPL

In [ ]:
dshift() #DOPPLER computed value, TaylorIntegration.jl, with A2=0

In [ ]:
apophis_data[rad_ind].doppler - dshift() #DELAY O-C, TaylorIntegration, with A2=0

In [ ]:
apophis_data[rad_ind].doppler - dshift(-5.762) #DELAY O-C, TaylorIntegration, with A2=-5.7

In [ ]:
#compute time-delay and Doppler-shift "ephemeris" (i.e., predicted values according to ephemeris)
tdelay_v1, dshift_v1 = delay_doppler(apophis_data, niters; xve=earth_et, xvs=sun_et, xva=apophis_et);

In [ ]:
# compute time-delay, Doppler-shift values from JPL ephemerides
tdelay_JPL_v1, dshift_JPL_v1 = delay_doppler(apophis_data);

In [ ]:
tdelay_jpl_obs = [x.delay for x in apophis_data][delay_index]
dshift_jpl_obs = [x.doppler for x in apophis_data][doppler_index]

tdelay_jpl_obs_sigma = [x.delay_sigma for x in apophis_data][delay_index]
dshift_jpl_obs_sigma = [x.doppler_sigma for x in apophis_data][doppler_index];

In [ ]:
# Observed minus computed (O-C) residuals: JPL DE430+s199

# absolute residuals
residual_JPL_td_ = tdelay_jpl_obs - tdelay_JPL_v1  # (usec)
residual_JPL_ds = dshift_jpl_obs - (dshift_JPL_v1) # (Hz)

# linear correction to delays:
residual_JPL_td = residual_JPL_td_ #.+ (0.2e-2(tv_jpl_utc_jul[delay_index].-tv_jpl_utc_jul[end])) # (usec)

# relative residuals
rel_res_JPL_td = residual_JPL_td ./ tdelay_jpl_obs # (usec)
rel_res_JPL_ds = residual_JPL_ds ./ dshift_jpl_obs # (Hz)
;

In [ ]:
#Observed minus computed (O-C) residuals: TaylorIntegration.jl

#absolute
residual_td = tdelay_jpl_obs - (tdelay_v1)  # (usec)
residual_ds = dshift_jpl_obs - (dshift_v1) # (Hz)

#relative
rel_res_td = residual_td ./ tdelay_jpl_obs # (usec)
rel_res_ds = residual_ds ./ dshift_jpl_obs # (Hz)
;

## Empirical correction to observation ephemeris vs JPL sol. #199

In [ ]:
###time-delay and Doppler residuals from Brozovic et al., 2018
b2018_td_res = [0.105, 0.023, -0.491, 0.018, -0.123, -0.599, 0.121, 0.072, 0.056, 0.021, 0.059, -0.374, -0.154, 0.049, 0.400]
b2018_ds_res = [0.039, -0.023, -0.001, 0.107, -0.061, -0.052, 0.037, -0.009, -0.007, -0.027, 0.031, 0.113, -0.062, 0.010, -0.077, 0.070, -0.006, 0.009, -0.068, 0.004, 0.023, 0.016, 0.029, 0.150]
;

In [ ]:
#residual_JPL_td

In [ ]:
plot(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_JPL_td.-b2018_td_res,
    #yerror = tdelay_jpl_obs_sigma
    m=:cross
)

In [ ]:
#residual_JPL_ds

In [ ]:
plot(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds.-b2018_ds_res,
    m=:cross
)

# Time-delay / Doppler shift plots

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[delay_index].-t0, tdelay_v1(A2_plot),
    label="predicted (TaylorIntegration, A2 = $A2_plot x 10^-14 au/d^2)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_JPL_v1,
    label="predicted (JPL DE430+s199)",
    marker=:xcross
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_jpl_obs,
    label="observed (Goldstone, Arecibo 2012-2013)",
    legend=:topright,
    marker=:circle,
    markersize=3,
    yerror=dshift_jpl_obs_sigma
)
title!("Time delay vs time")
ylabel!("Total time delay [useconds]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[doppler_index].-t0, dshift_v1(A2_plot),
    label="predicted (TaylorIntegration, A2 = $A2_plot x 10^-14 au/d^2)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_JPL_v1,
    label="predicted (JPL DE430+s199)",
    marker=:xcross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_jpl_obs,
    label="observed (Goldstone, Arecibo 2012-2013)",
    legend=:topright,
    marker=:circle,
    markersize=3,
    yerror=dshift_jpl_obs_sigma
)
title!("Doppler shift vs time")
ylabel!("Total Doppler shift [Hz]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
#A2_lsqfit_deldop=-5.757903669935907

In [ ]:
#A2_plot = -5.592839897872 # JPL solution #199
#A2_plot = -5.757903669935907 # A2_lsqfit_deldop
A2_plot = 0.0 #-5.7
A2_label_str = @sprintf "TaylorIntegration (A2 = %.2f x 10^-14 au/d^2)"  A2_plot
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(A2_plot), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = A2_label_str,
    legend = :topright
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_JPL_td, yerror = tdelay_jpl_obs_sigma,
    marker=:xcross,
    label = "JPL DE430+s199",
    legend = :topright
)
title!("Time delay residuals (O-C)")
ylabel!("Total time delay resid. (O-C) [us]")
xlims!(1540, 1645)
#ylims!(-215,210)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
#savefig("delay_residuals_A2_0.pdf")
#savefig("delay_residuals_A2_lsfit.pdf")

In [ ]:
A2_lsqfit_deldop
# -5.757903669935907 # with weights

In [ ]:
#A2_plot = -5.592839897872 # JPL solution #199
#A2_plot = -5.757903669935907 # A2_lsqfit_deldop
A2_plot = -5.757903669935907
#A2_plot = 0.0 #-5.7
A2_label_str = @sprintf "TaylorIntegration (A2 = %.2fx10^-14 au/d^2)"  A2_plot
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(A2_plot), yerror = tdelay_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = A2_label_str,
    legend = :topright
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_JPL_td, yerror = tdelay_jpl_obs_sigma,
    marker=:circle,
    ms=3.0,
    label = "JPL DE430+s199",
    legend = :bottomright
)
title!("Time delay residuals (O-C)")
ylabel!("Total time delay resid. (O-C) [us]")
xlims!(1540, 1645)
#ylims!(-5,4)
#ylims!(-.5,.5)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
#savefig("delay_residuals_A2_0.pdf")
#savefig("delay_residuals_A2_lsfit.pdf")

In [ ]:
#A2_plot = -5.592839897872 # JPL solution #199
#A2_plot = -5.757903669935907 # A2_lsqfit_deldop
A2_plot = 0.0 #-5.7
A2_label_str = @sprintf "TaylorIntegration (A2 = %.2fx10^-14 au/d^2)"  A2_plot
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(A2_plot), yerror = dshift_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = A2_label_str,
    leg=false
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds, yerror = dshift_jpl_obs_sigma,
    marker=:circle,
    ms=3.0,
    label = "JPL DE430+s199",
    legend=:bottomleft
)
title!("Doppler shift residuals (O-C)")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
#ylims!(-1.0,1.2)
#savefig("doppler_residuals_A2_0.pdf")
#savefig("doppler_residuals_A2_lsfit.pdf")

In [ ]:
#A2_plot = -5.592839897872 # JPL solution #199
A2_plot = -5.757903669935907 # A2_lsqfit_deldop
#A2_plot = 0.0 #-5.7
A2_label_str = @sprintf "TaylorIntegration (A2 = %.2fx10^-14 au/d^2)"  A2_plot
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(A2_plot), yerror = dshift_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = A2_label_str,
    leg=false
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds, yerror = dshift_jpl_obs_sigma,
    marker=:circle,
    ms=3.0,
    label = "JPL DE430+s199",
    legend=:bottomleft
)
title!("Doppler shift residuals (O-C)")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
#ylims!(-0.6,0.5)
#ylims!(-1.0,1.2)
#savefig("doppler_residuals_A2_0.pdf")
#savefig("doppler_residuals_A2_lsfit.pdf")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0, residual_JPL_td, yerror = tdelay_jpl_obs_sigma,
    marker=:xcross,
    label = "JPL DE430+s199",
    legend = :topright,
    color=2
)
ylabel!("Total time delay resid. (O-C) [us]")
#xlims!(1540, 1645)
#xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
title!("Time delay residuals (O-C)")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds, yerror = dshift_jpl_obs_sigma,
    marker=:xcross,
    label = "JPL DE430+s199",
    legend=:bottomright,
    color=2
)
title!("Doppler shift residuals (O-C)")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
plot(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_JPL_td./tdelay_jpl_obs_sigma,
    #yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="JPL DE430+s199",
    marker=:xcross,
    legend=:topright,
    color=2
)
title!("Time delay Nresid. (O-C)/sigma")
xlims!(1540, 1645)
#ylims!(-1.1,1.1)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
ylabel!("Normalized residual")

In [ ]:
plot(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds./dshift_jpl_obs_sigma,
    #yerror = dshift_jpl_obs_sigma./dshift_jpl_obs,
    label="JPL DE430+s199",
    legend=:bottomright,
    marker=:xcross,
    color=2
)
title!("Doppler shift Nresid. (O-C)/sigma")
xlims!(1540, 1645)
#ylims!(-2, 2)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

A2_plot = 0.0 #A2_lsqfit_del
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    rel_res_td(A2_plot),
    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="TaylorIntegration (A2 = $A2_plot x 10^-14 au/d^2)",
    marker=:cross,
    legend=:topright
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    rel_res_JPL_td,
    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="JPL DE430+s199",
    marker=:xcross,
    legend=:topright
)
title!("Time delay relative resid. (O-C)/O")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    rel_res_ds(A2_plot),
    yerror = dshift_jpl_obs_sigma./dshift_jpl_obs,
    label="TaylorIntegration (A2 = $A2_plot x 10^-14 au/d^2)",
    legend=:bottomright,
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    rel_res_JPL_ds,
    yerror = dshift_jpl_obs_sigma./dshift_jpl_obs,
    label="JPL DE430+s199",
    legend=:bottomright,
    marker=:xcross
)
title!("Doppler shift relative resid. (O-C)/O")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

#check that HORIZONS (s#199) and downloaded SPK file for Apophis are the same
#according to this check, they differ at the end of the evaluated times by ~ 2m in position
tv_spkvhor = range(2454733.5, length=1001, stop=2456367.5)
apophis_spk_arr = Array{Float64}(undef, length(tv_spkvhor), 6)
for i in eachindex(tv_spkvhor)
    apophis_spk_arr[i,:] .= Apophis.apophis_pv(tv_spkvhor[i])
end
apophis_horizons_arr = readdlm("../wld103954.15")
@show norm((apophis_spk_arr .- apophis_horizons_arr)[end,1:3])
@show norm((apophis_spk_arr .- apophis_horizons_arr)[end,4:6])
;

# Yarkovsky $A_2$ coefficient estimation

Here, we will compute, for a single time delay observation, the value of $A_2$ which makes the O-C residual of that observation equal to zero:

The observed value is:

In [ ]:
#observed value
tdelay_jpl_obs[1]

In [ ]:
# computed value (full solution)
tdelay_v1[delay_index[1]] # τ_c(A2) = p(A2)

In [ ]:
# residual (A2 = 0)
tdelay_jpl_obs[1] - tdelay_v1[delay_index[1]]()

In [ ]:
# substract A2=0 solution from τ_c(A2) (i.e., compute τ_c(A2)-τ_c(A2=0) )
p_τ_1 = tdelay_v1[delay_index[1]]-tdelay_v1[delay_index[1]]()

In [ ]:
# Let p_τ_1(A2) be the above polynomial expression, then we invert it so we get A2(δτ):
A2_δτ_1 = inverse(p_τ_1)

In [ ]:
#Now, we evaluate the polynomial at the corresponding residual at the A2=0 TaylorIntegration solution:
A2_δτ_1(residual_td[1]())

Now, we follow the same strategy for all the 2012-2013 radar astrometry for Apohpis (delay and Doppler):

In [ ]:
td_us = tdelay_v1
ds_Hz = dshift_v1;

In [ ]:
A2_δτ_v = (  inverse.( td_us-td_us() )  ) # vector of A2(δτ) polynomials at each delay observation;
A2_δν_v = (  inverse.( ds_Hz-ds_Hz() )  ) # vector of A2(δν) polynomials at each Doppler observation;

In [ ]:
A2_del_v = map((x,y)->x(y), A2_δτ_v, residual_td()); # A2(δτ) polynomials evaluated at the O-C time delay residuals;
A2_dop_v = map((x,y)->x(y), A2_δν_v, residual_ds()); # A2(δν) polynomials evaluated at the O-C Doppler shift residuals;

In [ ]:
A2_del_v

In [ ]:
A2_dop_v

In [ ]:
mean(A2_del_v), std(A2_del_v)

In [ ]:
mean(A2_dop_v), std(A2_dop_v)

In [ ]:
A2_all = vcat(A2_del_v, A2_dop_v)

In [ ]:
A2_mean_all = mean(A2_all)
A2_std_all = std(A2_all)
A2_mean_all, A2_std_all

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    A2_del_v[1:end-3],
    marker=:cross,
    label="A2 est. (Goldstone delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    A2_del_v[end-2:end],
    marker=:xcross,
    label="A2 est. (Arecibo delay obs.)",
    color=3,
    leg=:topright
)
ylabel!("A2(dt) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    A2_dop_v[1:end-7],
    marker=:cross,
    label="A2 est. (Goldstone Doppler obs.)",
    color=2
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    A2_dop_v[end-6:end],
    marker=:xcross,
    label="A2 est. (Arecibo Doppler obs.)",
    leg=:bottomleft,
    color=4
)
ylabel!("A2(df) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    A2_del_v[1:end-3],
    marker=:cross,
    label="A2 est. (Goldstone delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    A2_dop_v[1:end-7],
    marker=:cross,
    label="A2 est. (Goldstone Doppler obs.)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    A2_del_v[end-2:end],
    marker=:xcross,
    label="A2 est. (Arecibo delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    A2_dop_v[end-6:end],
    marker=:xcross,
    label="A2 est. (Arecibo Doppler obs.)",
    leg=:bottomleft
)
ylabel!("A2 x 10^14 [au/d^2]")
xlims!(1540, 1645)
ylims!(-105,5)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_NO_Arecibo_dop = vcat(A2_del_v, A2_dop_v[1:end-7]);

In [ ]:
A2_mean_NO_Arecibo_dop = mean(A2_NO_Arecibo_dop)
A2_std_NO_Arecibo_dop = std(A2_NO_Arecibo_dop)
A2_mean_NO_Arecibo_dop, A2_std_NO_Arecibo_dop

# Intervals

In [ ]:
#]add IntervalArithmetic

In [ ]:
using IntervalArithmetic

In [ ]:
residual_td_interval = interval.(
    (tdelay_jpl_obs-tdelay_jpl_obs_sigma)-tdelay_v1(), 
    (tdelay_jpl_obs+tdelay_jpl_obs_sigma)-tdelay_v1()
);

In [ ]:
residual_ds_interval = interval.(
    (dshift_jpl_obs-dshift_jpl_obs_sigma)-dshift_v1(),
    (dshift_jpl_obs+dshift_jpl_obs_sigma)-dshift_v1()
);

In [ ]:
A2_del_v_interval = map((x,y)->x(y), A2_δτ_v, residual_td_interval)

In [ ]:
A2_dop_v_interval = map((x,y)->x(y), A2_δν_v, residual_ds_interval)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    label="delay (Goldstone)",
    leg=:topright,
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    label="delay (Arecibo)",
    leg=:topright,
    marker=:xcross,
    color=3
)
#ylims!(minimum(A2_del_v)-1,0)
ylabel!("A2(dt) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
title!("A2 estimation from polynomial inversion of delay obs.")
#savefig("A2_poly_inv_delay.pdf")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    label="Doppler (Goldstone)",
    marker=:cross,
    color=2
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    label="Doppler (Arecibo)",
    marker=:xcross,
    leg=:bottomleft,
    color=4
)
ylabel!("A2(df) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_v_NO_Arecibo_dop_interval = vcat(A2_del_v_interval, A2_dop_v_interval[1:end-7])

In [ ]:
A2_mean_NO_Arecibo_dop_interval = mean(A2_v_NO_Arecibo_dop_interval)
A2_std_NO_Arecibo_dop_interval = std(A2_v_NO_Arecibo_dop_interval)
A2_mean_NO_Arecibo_dop_interval, A2_std_NO_Arecibo_dop_interval

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    marker=:cross,
    leg=:bottomleft,
    label="delay (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    marker=:cross,
    label="Doppler (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    marker=:xcross,
    leg=:bottomleft,
    label="delay (Arecibo)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    marker=:xcross,
    label="Doppler (Arecibo)"
)
A2_label_str = @sprintf "A2 (fit): %.5f  x 10^-14 au/d^2" A2_lsqfit_deldop
plot!([1540, 1640], [A2_lsqfit_deldop, A2_lsqfit_deldop], label=A2_label_str)
ylabel!("A2 x 10^14 [au/day^2]")
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
title!("A2 estimation from radar astrometry (Brozovic et al., 2018)")
#savefig("A2_fit_all.pdf")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    marker=:cross,
    leg=:bottomleft,
    label="delay (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    marker=:cross,
    label="Doppler (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    marker=:xcross,
    leg=:bottomright,
    label="delay (Arecibo)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    marker=:xcross,
    label=""#"Doppler (Arecibo)"
)
A2_label_str = @sprintf "A2 (fit): %.5f  x 10^-14 au/d^2" A2_lsqfit_deldop_NO_Arecibo_dop
plot!([1540, 1645], [A2_lsqfit_deldop, A2_lsqfit_deldop], label=A2_label_str)
ylims!(-20,0)
ylabel!("A2 x 10^14 [au/day^2]")
xlims!(1540, 1645)
xlabel!("Time")
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1645)))))
title!("A2 estimation from radar astrometry (Brozovic et al., 2018)")
#savefig("A2_fit_no_Arecibo_doppler.pdf")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    marker=:cross,
    leg=:bottomleft,
    label="delay (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    marker=:cross,
    label="Doppler (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    marker=:xcross,
    leg=:bottomright,
    label="delay (Arecibo)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    marker=:xcross,
    label=""#"Doppler (Arecibo)"
)
A2_label_str = @sprintf "A2 (fit): %.5f  x 10^-14 au/d^2" A2_lsqfit_deldop_NO_Arecibo_dop
plot!([1540, 1645], [A2_lsqfit_del, A2_lsqfit_del], label=A2_label_str)
ylims!(-8,-4)
ylabel!("A2 x 10^14 [au/day^2]")
xlims!(1540, 1645)
xlabel!("Time")
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1645)))))
title!("A2 estimation from radar astrometry (Brozovic et al., 2018)")
#savefig("A2_fit_no_Arecibo_doppler.pdf")

## Least squares

In [ ]:
res_del = tdelay_jpl_obs .- tdelay_v1 # delay residuals a.a.f. of A2
res_dop = dshift_jpl_obs .- dshift_v1 # Doppler residuals a.a.f. of A2
W_del = 1 ./(tdelay_jpl_obs_sigma.^2) # delay weights
W_dop = 1 ./(dshift_jpl_obs_sigma.^2) # Doppler weights
res_deldop = vcat(res_del, res_dop) # delay + Doppler residuals as a function of A2
W_deldop = vcat(W_del, W_dop) # delay + Doppler weights
res_xWx_del = res_del .* W_del .* res_del
res_xWx_dop = res_dop .* W_dop .* res_dop
res_xWx_deldop = res_deldop .* W_deldop .* res_deldop
res_xWx_deldop_NO_Arecibo_dop = vcat(res_xWx_del, res_xWx_dop[1:end-7]); #remove Arecibo Doppler "outliers"

In [ ]:
Apophis.taylor_roots.(res_del, 0.0)

scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    taylor_roots.(res_del, 0.0),
    yerror=radius.(A2_del_v_interval)
)

Below, we compute the $A_2$ value which minimizes the polynomial expression for $Q(A_2)$, the mean square residual. We consider 4 cases: 1) only the delay observations, 2) only the Doppler observations, 3) all the delays and Dopplers, and 4) all delays and Dopplers excluding Arecibo Dopplers. Further, following classical nonlinear squares procedures, we estimate the 1-$\sigma$ formal uncertainty of $A_2$ for each case, from the covariance "matrix" as $\sigma_{A_2} = \Gamma_{1,1}$ where the covariance matrix is $\Gamma = C^{-1}$, $C$ is the normal matrix $C = (\partial\xi/\partial A_2)^\dagger \cdot (\partial\xi/\partial A_2)$ and $\xi$ are the residuals. In this case, since were are fitting only 1 parameter, namely $A_2$, both $\Gamma$ and $C$ correspond to 1x1 matrices (i.e., both $\Gamma$ and $C$ are scalars).

In [ ]:
Q_A2_del = sum(res_xWx_del)/length(res_xWx_del)
#res_sq_del_ord2 = map(x->Taylor1(x.coeffs[1:3]), res_sq_del)
#Q_A2_del = sum(res_sq_del_ord2)/length(res_sq_del_ord2)

In [ ]:
A2_lsqfit_del = Apophis.taylor_minimum(Q_A2_del, 0.0, niters=5)

In [ ]:
#B_del = differentiate.( map(x->Taylor1(x.coeffs[1:3]), res_del) )
B_del = differentiate.(res_del)
C_del = ( transpose(B_del) )*( W_del.*B_del )
Γ_del = inv(C_del)

In [ ]:
Γ_del(A2_lsqfit_del)

In [ ]:
sqrt( Γ_del(A2_lsqfit_del) )

In [ ]:
Q_A2_dop = sum(res_xWx_dop)/length(res_xWx_dop)

In [ ]:
A2_lsqfit_dop = Apophis.taylor_minimum(Q_A2_dop, 0.0, niters=5)

In [ ]:
B_dop = differentiate.(res_dop)
C_dop = ( transpose(B_dop) )*( W_dop.*B_dop )
Γ_dop = inv(C_dop)

In [ ]:
Γ_dop(A2_lsqfit_dop)

In [ ]:
sqrt( Γ_dop(A2_lsqfit_dop) )

In [ ]:
Q_A2_deldop = sum(res_xWx_deldop)/length(res_xWx_deldop)

In [ ]:
A2_lsqfit_deldop = Apophis.taylor_minimum(Q_A2_deldop, 0.0, niters=5)

In [ ]:
B_deldop = differentiate.(res_deldop)
C_deldop = ( transpose(B_deldop) )*( W_deldop.*B_deldop )
Γ_deldop = inv(C_deldop)

In [ ]:
Γ_deldop(A2_lsqfit_deldop)

In [ ]:
sqrt( Γ_deldop(A2_lsqfit_deldop) )

In [ ]:
Q_A2_deldop_NO_Arecibo_dop = sum(res_xWx_deldop_NO_Arecibo_dop)/length(res_xWx_deldop_NO_Arecibo_dop)

In [ ]:
A2_lsqfit_deldop_NO_Arecibo_dop = Apophis.taylor_minimum(Q_A2_deldop_NO_Arecibo_dop, 0.0, niters=5)

In [ ]:
Γ_deldop(A2_lsqfit_deldop_NO_Arecibo_dop)

In [ ]:
sqrt( Γ_deldop(A2_lsqfit_deldop_NO_Arecibo_dop) )

In [ ]:
gr()

In [ ]:
_A2_min_plot = -5.85
_A2_max_plot = -5.7
A2_vals = range(_A2_min_plot, length=2000, stop=_A2_max_plot)
plot(A2_vals, Q_A2_deldop.(A2_vals), leg=false)
xlabel!("A2 x 10^14 [au/day^2]")
ylabel!("Q(A2)")
title!("Mean weighted square residual vs A2")
#ylims!(-1, 11)
xlims!(_A2_min_plot, _A2_max_plot)

In [ ]:
mean(res_deldop), std(res_deldop)

Compute the value of A2 such that the mean of the residuals is zero:

In [ ]:
A2_zero_mean_res = Apophis.taylor_roots(mean(res_deldop), A2_lsqfit_deldop)

Check that the vector of residuals evaluated at the LS solution has mean zero:

In [ ]:
mean(res_deldop()), std(res_deldop()) #A2 = 0

In [ ]:
# A2 = A2 LS solution
mean(res_deldop(A2_lsqfit_deldop)), std(res_deldop(A2_lsqfit_deldop))

In [ ]:
# A2 such that res. mean is closest to zero
mean(res_deldop(A2_zero_mean_res)), std(res_deldop(A2_zero_mean_res))

If we expand the "cost" function Q wrt the solution value, then the first-order term should be zero:

In [ ]:
ΔQ_del = Q_A2_del(A2_lsqfit_del+Taylor1(Q_A2_del.order))-Q_A2_del(A2_lsqfit_del)

In [ ]:
ΔQ_deldop = Q_A2_deldop(A2_lsqfit_deldop+Taylor1(Q_A2_deldop.order))-Q_A2_deldop(A2_lsqfit_deldop)

- Compare directly ephemerides JPL vs TaylorIntegration
- Propagate A2 interval trough March 2021 approach to see what difference in us, hz expect from A2 not zero

## Ephemerides: DE430+s199 vs TaylorIntegration

In [ ]:
tv

In [ ]:
(tv[end]-tv[1])/365.25

In [ ]:
tv_et = Array{Float64}(undef, length(tv))
apophis_jpl = Array{Float64}(undef, length(tv), 6)
earth_jpl = Array{Float64}(undef, length(tv), 6)
moon_jpl = Array{Float64}(undef, length(tv), 6)
for i in eachindex(tv)
    tv_et[i] = julian2etsecs(tv[i])
    apophis_jpl[i,:] .= Apophis.apophis_pv_199(tv_et[i])
    earth_jpl[i,:] .= Apophis.earth_pv(tv_et[i])
    moon_jpl[i,:] .= Apophis.moon_pv(tv_et[i])
end

In [ ]:
#A2_lsqfit_deldop = A2_lsqfit_del
apophis_ti = [apophis(tv[i])[j](A2_lsqfit_deldop) for i in eachindex(tv), j in 1:6];
earth_ti = [ss16asteph(tv[i])[j] for i in eachindex(tv), j in union(3*4-2:3*4,3*(27+4)-2:3*(27+4))];
moon_ti = [ss16asteph(tv[i])[j] for i in eachindex(tv), j in union(3*5-2:3*5,3*(27+5)-2:3*(27+5))];

In [ ]:
1000*au*(apophis_ti - apophis_jpl) 

In [ ]:
1000au*(earth_ti - earth_jpl)

In [ ]:
1000au*(moon_ti - moon_jpl)

In [ ]:
x_ae_ti = apophis_ti - earth_ti;

In [ ]:
x_ae_jpl = apophis_jpl - earth_jpl;

In [ ]:
r(x,y,z) = sqrt(x^2 + y^2 + z^2)
vr(x,y,z,u,v,w) = (x*u + y*v + z*w)/r(x,y,z)

r_ae_ti = r.(x_ae_ti[:,1], x_ae_ti[:,2], x_ae_ti[:,3])
vr_ae_ti = vr.( x_ae_ti[:,1], x_ae_ti[:,2], x_ae_ti[:,3], x_ae_ti[:,4], x_ae_ti[:,5], x_ae_ti[:,6] )

r_ae_jpl = r.(x_ae_jpl[:,1], x_ae_jpl[:,2], x_ae_jpl[:,3])
vr_ae_jpl = vr.( x_ae_jpl[:,1], x_ae_jpl[:,2], x_ae_jpl[:,3], x_ae_jpl[:,4], x_ae_jpl[:,5], x_ae_jpl[:,6] );

In [ ]:
#tve = range(tv[1], tv[end], length=2000)
tvemin, tvemax = minmax(ss16asteph.t[1], ss16asteph.t[end])
tve = range(tvemin, tvemax, length=50000)#2000)
#tve = range(ss16asteph.t[1], ss16asteph.t[1]+yr, length=50000)

In [ ]:
julian2datetime.(tve)

In [ ]:
sun_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*1-2:3*1,3*(27+1)-2:3*(27+1))]
sun_jpl_e = Array{Float64}(undef, length(tve), 6)
mercury_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*2-2:3*2,3*(27+2)-2:3*(27+2))]
mercury_jpl_e = Array{Float64}(undef, length(tve), 6)
venus_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*3-2:3*3,3*(27+3)-2:3*(27+3))]
venus_jpl_e = Array{Float64}(undef, length(tve), 6)
earth_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*4-2:3*4,3*(27+4)-2:3*(27+4))]
earth_jpl_e = Array{Float64}(undef, length(tve), 6)
moon_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*5-2:3*5,3*(27+5)-2:3*(27+5))]
moon_jpl_e = Array{Float64}(undef, length(tve), 6)
mars_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*6-2:3*6,3*(27+6)-2:3*(27+6))]
mars_jpl_e = Array{Float64}(undef, length(tve), 6)
jupiter_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*7-2:3*7,3*(27+7)-2:3*(27+7))]
jupiter_jpl_e = Array{Float64}(undef, length(tve), 6)
saturn_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*8-2:3*8,3*(27+8)-2:3*(27+8))]
saturn_jpl_e = Array{Float64}(undef, length(tve), 6)
for i in eachindex(tve)
    tve_et_i = julian2etsecs(tve[i])
    sun_jpl_e[i,:] .= Apophis.sun_pv(tve_et_i)
    mercury_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(1, 0, tve_et_i) )
    venus_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(2, 0, tve_et_i) )
    earth_jpl_e[i,:] .= Apophis.earth_pv(tve_et_i)
    moon_jpl_e[i,:] .= Apophis.moon_pv(tve_et_i)
    mars_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(4, 0, tve_et_i) )
    jupiter_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(5, 0, tve_et_i) )
    saturn_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(6, 0, tve_et_i) )
end

In [ ]:
#using SPICE

In [ ]:
#furnsh("../jpleph/a99942_s199.bsp")

In [ ]:
#apophis_pv_199(et) = Apophis.kmsec2auday( Apophis.getpv(2099942, 0, et) ) # units: au, au/day

In [ ]:
apophis_ti_e = [apophis(tve[i])[j]( A2_lsqfit_deldop ) for i in eachindex(tve), j in 1:6];
apophis_jpl_e = Array{Float64}(undef, length(tve), 6)
for i in eachindex(tve)
    tve_et_i = julian2etsecs(tve[i])
    apophis_jpl_e[i,:] .= Apophis.apophis_pv_199(tve_et_i)
end

In [ ]:
gr()

In [ ]:
length(tve[1]:20:tve[end])

In [ ]:
plot( tve, 1000au*(sun_ti_e .- sun_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(sun_ti_e .- sun_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(sun_ti_e .- sun_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Sun: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, (1000/86400)*au*(sun_ti_e .- sun_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(sun_ti_e .- sun_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(sun_ti_e .- sun_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Sun: diff. in barycentric velocity vs DE430")
ylabel!("PlEph.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
#plotlyjs()
gr()

In [ ]:
plot( tve, 1000au*(mercury_ti_e .- mercury_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(mercury_ti_e .- mercury_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(mercury_ti_e .- mercury_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Mercury: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, (1000/86400)*au*(mercury_ti_e .- mercury_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(mercury_ti_e .- mercury_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(mercury_ti_e .- mercury_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Mercury: diff. in barycentric velocity vs DE430")
ylabel!("PlEph.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, 1000au*(venus_ti_e .- venus_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(venus_ti_e .- venus_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(venus_ti_e .- venus_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Venus: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, (1000/86400)*au*(venus_ti_e .- venus_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(venus_ti_e .- venus_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(venus_ti_e .- venus_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Venus: diff. in barycentric velocity vs DE430")
ylabel!("PlEph.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot(tve, 1000au*(earth_ti_e .- earth_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(earth_ti_e .- earth_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(earth_ti_e .- earth_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Earth: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, (1000/86400)*au*(earth_ti_e .- earth_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(earth_ti_e .- earth_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(earth_ti_e .- earth_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Earth: diff. in barycentric velocity vs DE430")
ylabel!("PlEph.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, 1000au*(moon_ti_e .- moon_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(moon_ti_e .- moon_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(moon_ti_e .- moon_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Moon: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, (1000/86400)*au*(moon_ti_e .- moon_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(moon_ti_e .- moon_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(moon_ti_e .- moon_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Moon: diff. in barycentric velocity vs DE430")
ylabel!("PlEph.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, 1000au*(mars_ti_e .- mars_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(mars_ti_e .- mars_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(mars_ti_e .- mars_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Mars: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, (1000/86400)*au*(mars_ti_e .- mars_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(mars_ti_e .- mars_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(mars_ti_e .- mars_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Mars: diff. in barycentric velocity vs DE430")
ylabel!("PlEph.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
gr()

In [ ]:
plot( tve, 1000au*(jupiter_ti_e .- jupiter_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(jupiter_ti_e .- jupiter_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(jupiter_ti_e .- jupiter_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Jupiter: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, (1000/86400)*au*(jupiter_ti_e .- jupiter_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(jupiter_ti_e .- jupiter_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(jupiter_ti_e .- jupiter_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Jupiter: diff. in barycentric velocity vs DE430")
ylabel!("PlEph.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, 1000au*(saturn_ti_e .- saturn_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(saturn_ti_e .- saturn_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(saturn_ti_e .- saturn_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Saturn: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
plot( tve, (1000/86400)*au*(saturn_ti_e .- saturn_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(saturn_ti_e .- saturn_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(saturn_ti_e .- saturn_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Saturn: diff. in barycentric velocity vs DE430")
ylabel!("PlEph.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
xlims!(tvemin, tvemax)
xticks!(tvemin:1200:tvemax, string.(Date.(julian2datetime.(tvemin:1200:tvemax))))

In [ ]:
gr()
plotlyjs()

In [ ]:
plot( tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#199")
ylabel!("Apophis.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
#xlims!(tve[1], tve[end-80])
#ylims!(-1000, 1000)
#xticks!(tve[1]:1200:tve[end-80], string.(Date.(julian2datetime.(tve[1]:1200:tve[end-80]))))

In [ ]:
plot( tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#199")
ylabel!("Apophis.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
#ylims!(-1000, 1000)
xlims!(tvemin, tvemax)
xticks!(tvemin:365:tvemax, string.(Date.(julian2datetime.(tvemin:365:tvemax))))

In [ ]:
tvemin:365:tvemax

In [ ]:
ans[2]

In [ ]:
#gr()
plotly()

In [ ]:
plot( tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#199")
ylabel!("Apophis.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
#xlims!(tve[end-2900], tve[end-2850])
#ylims!(-5e3, 5e3)
xticks!(tve[1]:1200:tve[end-80], string.(Date.(julian2datetime.(tve[1]:1200:tve[end-80]))))

In [ ]:
plot( tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#199")
ylabel!("Apophis.jl - DE430 [m]")
xlabel!("Julian date (TDB)")
#xlims!(tve[end-2900], tve[end-2850])
ylims!(-2.5e3, 2.5e3)
xticks!(tve[1]:1200:tve[end-80], string.(Date.(julian2datetime.(tve[1]:1200:tve[end-80]))))

In [ ]:
plot( tve, (1000/86400)*au*(apophis_ti_e .- apophis_jpl_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1000/86400)*au*(apophis_ti_e .- apophis_jpl_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1000/86400)*au*(apophis_ti_e .- apophis_jpl_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Apophis: diff. in barycentric velocity vs JPL#199")
ylabel!("Apophis.jl - DE430 [m/s]")
xlabel!("Julian date (TDB)")
#ylims!(-0.0002, 0.0002)
#xlims!(tve[1], tve[end-80])
#xticks!(tve[1]:1200:tve[end-80], string.(Date.(julian2datetime.(tve[1]:1200:tve[end-80]))))

In [ ]:
plot(tv[2:end], 1e6*86400(r_ae_ti[2:end]-r_ae_jpl[2:end])/c_au_per_day, m=:cross)
ylabel!("range diff [light-microseconds]")
xlabel!("Julian date")

In [ ]:
plot(tv[2:end], 1e6(vr_ae_jpl[2:end]-vr_ae_ti[2:end])/c_au_per_day, m=:cross)
ylabel!("Doppler shift diff (Hz)")
xlabel!("Julian date")

In [ ]:
(x_ae_ti - x_ae_jpl)./x_ae_jpl

In [ ]:
plot( tv[2:end], 1000au*(earth_ti .- earth_jpl)[2:end,1], label="x")#, marker=:cross)
plot!(tv[2:end], 1000au*(earth_ti .- earth_jpl)[2:end,2], label="y")#, marker=:cross)
plot!(tv[2:end], 1000au*(earth_ti .- earth_jpl)[2:end,3], label="z")#, marker=:cross)
title!("Earth: diff. in barycentric position vs DE430")
ylabel!("PlEph.jl - DE430 [m]")
xlabel!("Julian date (TDB)")

In [ ]:
plot( tv[2:end], (x_ae_ti - x_ae_jpl)[2:end,1]*au*(1e6/299792.458), label="x")#, marker=:cross)
plot!(tv[2:end], (x_ae_ti - x_ae_jpl)[2:end,2]*au*(1e6/299792.458), label="y")#, marker=:cross)
plot!(tv[2:end], (x_ae_ti - x_ae_jpl)[2:end,3]*au*(1e6/299792.458), label="z")#, marker=:cross)
title!("Apophis: diff. in geocentric position vs DE430+s199")
ylabel!("[light-microseconds]")
xlabel!("Julian date (TDB)")

In [ ]:
plot( tv[2:end], (apophis_ti-apophis_jpl)[2:end,1]*1000au, label="x")#, marker=:cross)
plot!(tv[2:end], (apophis_ti-apophis_jpl)[2:end,2]*1000au, label="y")#, marker=:cross)
plot!(tv[2:end], (apophis_ti-apophis_jpl)[2:end,3]*1000au, label="z")#, marker=:cross)
title!("Apophis: diff. in barycentric position vs Solution #199")
ylabel!("[m]")
xlabel!("Julian date (TDB)")

In [ ]:
gr()

In [ ]:
myplot_tv = t[2:10:end]
myplot_xv = apophis.(myplot_tv)
plot(myplot_tv.-t[1], map(x->abs(x[1][0]), myplot_xv), yscale=:log10, label="x[0]", leg=false)
for i in 1:9
    plot!(myplot_tv.-t[1], map(x->abs(x[1][i]), myplot_xv), label="x[$i]")
end
plot!(myplot_tv.-t[1], map(x->abs(x[1][10]), myplot_xv), label="x[10]")
title!("x")

In [ ]:
myplot_tv = t[2:10:end]
myplot_xv = apophis.(myplot_tv)
r2v = map(x->x[1]^2+x[2]^2+x[3]^2, myplot_xv)
plot((myplot_tv.-t[1])/yr, map(x->abs(x[0]), r2v), yscale=:log10, label="x[0]", leg=false)
for i in 1:9
    plot!((myplot_tv.-t[1])/yr, map(x->abs(x[i]), r2v), label="x[$i]")
end
plot!((myplot_tv.-t[1])/yr, map(x->abs(x[10]), r2v), label="x[10]")
title!("r^2")

In [ ]:
# myplot_tv = t[2:10:end]
plot( myplot_tv.-t[1], map(x->abs(x[2][0]), apophis.(myplot_tv)), yscale=:log10, label="x[0]", leg=false)
plot!(myplot_tv.-t[1], map(x->abs(x[2][1]), apophis.(myplot_tv)), label="x[1]")
plot!(myplot_tv.-t[1], map(x->abs(x[2][2]), apophis.(myplot_tv)), label="x[2]")
plot!(myplot_tv.-t[1], map(x->abs(x[2][3]), apophis.(myplot_tv)), label="x[3]")
plot!(myplot_tv.-t[1], map(x->abs(x[2][4]), apophis.(myplot_tv)), label="x[4]")
title!("y")

In [ ]:
myplot_tv = t[2:10:end]
plot( myplot_tv.-t[1], map(x->abs(x[3][0]), apophis.(myplot_tv)), yscale=:log10, label="x[0]", leg=false)
plot!(myplot_tv.-t[1], map(x->abs(x[3][1]), apophis.(myplot_tv)), label="x[1]")
plot!(myplot_tv.-t[1], map(x->abs(x[3][2]), apophis.(myplot_tv)), label="x[2]")
plot!(myplot_tv.-t[1], map(x->abs(x[3][3]), apophis.(myplot_tv)), label="x[3]")
plot!(myplot_tv.-t[1], map(x->abs(x[3][4]), apophis.(myplot_tv)), label="x[4]")
title!("z")

In [ ]:
myplot_tv = t[2:10:end]
plot( myplot_tv.-t[1], map(x->abs(x[4][0]), apophis.(myplot_tv)), yscale=:log10, label="x[0]", leg=false)
plot!(myplot_tv.-t[1], map(x->abs(x[4][1]), apophis.(myplot_tv)), label="x[1]")
plot!(myplot_tv.-t[1], map(x->abs(x[4][2]), apophis.(myplot_tv)), label="x[2]")
plot!(myplot_tv.-t[1], map(x->abs(x[4][3]), apophis.(myplot_tv)), label="x[3]")
plot!(myplot_tv.-t[1], map(x->abs(x[4][4]), apophis.(myplot_tv)), label="x[4]")
title!("vx")

In [ ]:
myplot_tv = t[2:10:end]
plot( myplot_tv.-t[1], map(x->abs(x[5][0]), apophis.(myplot_tv)), yscale=:log10, label="x[0]", leg=false)
plot!(myplot_tv.-t[1], map(x->abs(x[5][1]), apophis.(myplot_tv)), label="x[1]")
plot!(myplot_tv.-t[1], map(x->abs(x[5][2]), apophis.(myplot_tv)), label="x[2]")
plot!(myplot_tv.-t[1], map(x->abs(x[5][3]), apophis.(myplot_tv)), label="x[3]")
plot!(myplot_tv.-t[1], map(x->abs(x[5][4]), apophis.(myplot_tv)), label="x[4]")
title!("vy")

In [ ]:
myplot_tv = t[2:10:end]
plot( myplot_tv.-t[1], map(x->abs(x[6][0]), apophis.(myplot_tv)), yscale=:log10, label="x[0]", leg=false)
plot!(myplot_tv.-t[1], map(x->abs(x[6][1]), apophis.(myplot_tv)), label="x[1]")
plot!(myplot_tv.-t[1], map(x->abs(x[6][2]), apophis.(myplot_tv)), label="x[2]")
plot!(myplot_tv.-t[1], map(x->abs(x[6][3]), apophis.(myplot_tv)), label="x[3]")
plot!(myplot_tv.-t[1], map(x->abs(x[6][4]), apophis.(myplot_tv)), label="x[4]")
title!("vz")

In [ ]:
dq = set_variables("ξ", order=1, numvars=7)

In [ ]:
tN = load("Apophis_jtN7_ord1_5yr.jld", "t")
xN = load("Apophis_jtN7_ord1_5yr.jld", "x")
apophisN = TaylorInterpolant(tN, xN)

In [ ]:
apophisN_et(et) = apophisN( etsecs2julian(et) )[1:6]

In [ ]:
86400*(datetime2julian(DateTime(2008,9,24)) - datetime2julian(DateTime(2000,1,1,12)))

In [ ]:
apophisN_et(2.754864e8+86400*100)

In [ ]:
apophisN(t0+100.0)

In [ ]:
dq[end][1][end]

In [ ]:
function (tinterp::TaylorInterpolant{T,U,2})(t::TaylorN{T}) where {T<:Real, U<:Number}
    t0 = constant_term(t)
    @assert tinterp.t[1] ≤ t0 ≤ tinterp.t[end] "Evaluation time outside range of interpolation"
    ind = findlast(x->x≤t0, tinterp.t)
    if ind == lastindex(tinterp.t)
        δt = t-tinterp.t[ind-1]
        return tinterp.x[ind-1,:](δt)
    elseif tinterp.t[ind] == t0
        return tinterp.x[ind,:]
    else
        δt = t-tinterp.t[ind]
        return tinterp.x[ind,:](δt)
    end
end

In [ ]:
rad_ind_ = 2
tdelay, dshift = delay_doppler(
    apophis_data[rad_ind_].rcvr,
    apophis_data[rad_ind_].utcepoch,
    apophis_data[rad_ind_].freq,
    niters,
    xve = earth_et,
    xvs = sun_et,
    xva = apophisN_et
)

In [ ]:
rad_ind_ = 2
tdelay, dshift = delay_doppler(
    apophis_data[rad_ind_].rcvr,
    apophis_data[rad_ind_].utcepoch,
    apophis_data[rad_ind_].freq,
    niters,
    xve = earth_et,
    xvs = sun_et,
    xva = apophis_et
)

In [ ]:
#get time-delay, Doppler-shift values from integration

tdelay_vN = Array{TaylorN{Float64}}(undef, length(apophis_data))
dshift_vN = Array{TaylorN{Float64}}(undef, length(apophis_data))

for i in eachindex(apophis_data)
    tdelay_vN[i], dshift_vN[i] = delay_doppler(
        apophis_data[i].rcvr,
        apophis_data[i].utcepoch,
        apophis_data[i].freq,
        niters,
        xve = earth_et,
        xvs = sun_et,
        xva = apophisN_et
    )
end

# observed values and uncertainties
tdelay_jpl_obs = [x.delay for x in apophis_data][delay_index]
dshift_jpl_obs = [x.doppler for x in apophis_data][doppler_index]

tdelay_jpl_obs_sigma = [x.delay_sigma for x in apophis_data][delay_index]
dshift_jpl_obs_sigma = [x.doppler_sigma for x in apophis_data][doppler_index];

#Observed minus computed (O-C) residuals: TaylorIntegration.jl

#absolute
residual_tdN = tdelay_jpl_obs - (tdelay_vN[delay_index])  # (usec)
residual_dsN = dshift_jpl_obs - (dshift_vN[doppler_index]) # (Hz)

#relative
rel_res_tdN = residual_tdN ./ tdelay_jpl_obs # (usec)
rel_res_dsN = residual_dsN ./ dshift_jpl_obs # (Hz)
;

In [ ]:
residual_tdN

## Propagation up to 2021/2038

In [ ]:
tve_2029 = range(
    datetime2julian(DateTime(2029,4,13,21,46)), 
    datetime2julian(DateTime(2029,4,13,21,47)), 
    length=3600
)

In [ ]:
A2_lsqfit_del

In [ ]:
apophis_ti_2029 = [apophis(tve_2029[i])[j](A2_lsqfit_del) for i in eachindex(tve_2029), j in 1:6];
#apophis_ti_2029 = [apophis(tve_2029[i])[j](-5.592839897872) for i in eachindex(tve_2029), j in 1:6];
earth_ti_2029 = [ss16asteph(tve_2029[i])[j] for i in eachindex(tve_2029), j in union(3*4-2:3*4,3*(27+4)-2:3*(27+4))];
x_ae_ti_2029 = apophis_ti_2029 .- earth_ti_2029

In [ ]:
r_ae_ti_2029 = r.(x_ae_ti_2029[:,1], x_ae_ti_2029[:,2], x_ae_ti_2029[:,3])
vr_ae_ti_2029 = vr.( x_ae_ti_2029[:,1], 
    x_ae_ti_2029[:,2], 
    x_ae_ti_2029[:,3], 
    x_ae_ti_2029[:,4], 
    x_ae_ti_2029[:,5], 
    x_ae_ti_2029[:,6] )

In [ ]:
ind_ca2029 = findfirst(x->x>0, vr_ae_ti_2029)

In [ ]:
vr_ae_ti_2029[ind_ca2029-1:ind_ca2029]

In [ ]:
r_ae_ti_2029[ind_ca2029], r_ae_ti_2029[ind_ca2029]*au

In [ ]:
0.0002521763836177017

In [ ]:
1000(0.0002521763836177017-0.000252172816142565)*au

In [ ]:
1000(r_ae_ti_2029[ind_ca2029]-0.000252172816142565)*au

In [ ]:
#JPL 2029 time of closest approach prediction (accurate to within a second)
Dates.julian2datetime( 2462240.40703 )

In [ ]:
Dates.julian2datetime( 2462240.407025 ), Dates.julian2datetime( 2462240.407034 )

In [ ]:
julian2datetime(tve_2029[ind_ca2029])

In [ ]:
julian2datetime(tve_2029[ind_ca2029-1])

In [ ]:
julian2datetime( (tve_2029[ind_ca2029-1]+tve_2029[ind_ca2029])/2 )

#recover integration from .jld file

#using JLD

vars = ["tv1", "xv1", "tvS1", "xvS1", "gvS1"] #names of variables
vars_2038 = ["tv1_2038", "xv1_2038", "tvS1_2038", "xvS1_2038", "gvS1_2038"] #names of variables
filename_2038 = string("Apophis_2038_jt.jld")
for i in eachindex(vars)
    ex_2038 = Symbol(vars_2038[i])
    @eval $ex_2038 = load($filename_2038, vars[$i])
end

tv1_2038

apophis_TI_diff = xv1_2038[:, Apophis.apophisdofs](A2_lsqfit_deldop)-xv1_2038[:, Apophis.apophisdofs]();

ae_TI_0 = xv1_2038[:, Apophis.apophisdofs]()-xv1_2038[:, Apophis.earthdofs]()
d_Ap_Ea_0 = sqrt.(ae_TI_0[:,1].^2 + ae_TI_0[:,2].^2 + ae_TI_0[:,3].^2)

ae_TI_A2 = xv1_2038[:, Apophis.apophisdofs](A2_lsqfit_deldop)-xv1_2038[:, Apophis.earthdofs]()
d_Ap_Ea_A2 = sqrt.(ae_TI_A2[:,1].^2 + ae_TI_A2[:,2].^2 + ae_TI_A2[:,3].^2)
;

gr()

_2021_approach = 3200:3263
plot(tv1_2038[_2021_approach].-tv1_2038[3200], (d_Ap_Ea_A2-d_Ap_Ea_0)[_2021_approach].*au)

findfirst(x->x>datetime2julian(DateTime(2021,1,1)), tv1_2038)

findlast(x->x<datetime2julian(DateTime(2021,4,1)), tv1_2038)

## Observations predictions 2021/2029

In [ ]:
apophis_data[end].rcvr

In [ ]:
typeof(ans)

In [ ]:
?delay_doppler

In [ ]:
tdelay_2021, dshift_2021 = delay_doppler(
    apophis_data[end].rcvr,
    DateTime(2021,1,1),
    apophis_data[end].freq,
    niters,
    false
)

In [ ]:
import EarthOrientation

In [ ]:
EarthOrientation.update()

## Plots for paper

In [ ]:
#gr()

In [ ]:
pgfplots(grid=:false, framestyle=:box)
font_size = 14

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[delay_index].-t0, tdelay_v1[delay_index](A2_plot),
    label="predicted (\$A_2 = $A2_plot \\times 10^{-14} \$ au/d\$^2\$)",
    marker=:cross,
    markersize=7,
)
#scatter!(
#    tv_jpl_utc_jul[delay_index].-t0,
#    tdelay_JPL_v1[delay_index],
#    label="predicted (JPL DE430+s199)",
#    marker=:xcross
#)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_jpl_obs,
    label="observed (Brozovic et al., 2018)",
    legend=:topleft,
    marker=:xcross,
    markersize=7,
    yerror=dshift_jpl_obs_sigma,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4),
    xticks = (1540:30:1640, ()),
    annotations=( 1640, 2.6e8, "(a)")
)
#title!("Time delay vs time")
ylabel!("Time delay [\$\\mu\$s]")
#xlims!(1540, 1645)
#xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlims!(1540, 1645)
ylims!(7.5e7,2.75e8)
#xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
#xlabel!("Time")

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[doppler_index].-t0, dshift_v1[doppler_index](A2_plot),
    label="predicted (\$A_2 = $A2_plot \\times 10^{-14} \$ au/d\$^2\$)",
    marker=:cross,
    markersize=7,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4)
)
#scatter!(
#    tv_jpl_utc_jul[doppler_index].-t0,
#    dshift_JPL_v1[doppler_index],
#    label="predicted (JPL DE430+s199)",
#    marker=:xcross
#)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_jpl_obs,
    label="observed (Brozovic et al., 2018)",
    legend=:bottomleft,
    marker=:xcross,
    markersize=7,
    yerror=dshift_jpl_obs_sigma,
    annotations=( 1640, 50000, "(b)")
)
title!("Doppler shift vs time")
ylabel!("Doppler shift [Hz]")
xlims!(1540, 1645)
xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
xlabel!("Time")

## Plots for DDA poster

In [ ]:
pgfplots(grid=:false, framestyle=:box)

In [ ]:
font_size = 14

In [ ]:
A2_plot = -5.74846133680956 # A2_lsqfit_deldop
A2_label_str = @sprintf "\$A_2^* = -5.74 \\times 10^{-14} \$ au/d\$^2\$"
plot(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = "\$A_2 = 0\$ au/d\$^2\$",
    legend = :topright,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4),
    xticks = (1540:30:1640, ()),
    annotations=( 1555, 300, "(a)")
)
for i in 1:4
    a2_lbl_str_i = "\$A_2 = -$i \\times 10^{-14}\$ au/d\$^2\$"
    plot!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(-i), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = a2_lbl_str_i,
    legend = :topright
)
end
plot!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(A2_plot), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = A2_label_str,
    legend = :topright
)
plot!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(-7), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = "\$A_2 = -7 \\times 10^{-14} \$ au/d\$^2\$",
    legend = :topright
)
ylabel!("Time delay O-C residual \$ [\\mu\\mathrm{s}] \$")
xlims!(1540, 1645)
#ylims!(-5,4)
ylims!(-250,350)
#xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
#xlabel!("Calendar date")
#savefig("delay_residuals_A2.pdf")

In [ ]:
A2_plot = -5.74846133680956 # A2_lsqfit_deldop
A2_label_str = @sprintf "\$A_2^* = -5.74846 \\times 10^{-14} \$ au/d\$^2\$"
plot(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = "\$A_2 = 0\$ au/d\$^2\$",
    legend = :bottomright,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4),
    annotations=( 1555, 0.95, "(b)")
)
for i in 3:3
    a2_lbl_str_i = "\$A_2 = -$i \\times 10^{-14} \$ au/d\$^2\$"
    plot!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(-i), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = a2_lbl_str_i,
    #legend = :topright,
    color=3+1
)
end
plot!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(A2_plot), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = A2_label_str,
    #legend = :topright,
    color=6
)
plot!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(-8), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = "\$A_2 = -7 \\times 10^{-14} \$ au/d\$^2\$",
    #legend = :topright,
    color=7
)
ylabel!("Doppler shift O-C residual \$ [\\mathrm{Hz}] \$")
xlims!(1540, 1645)
#ylims!(-5,4)
#ylims!(-250,350)
xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
xlabel!("Calendar date")
#savefig("doppler_residuals_A2.pdf")

In [ ]:
plot(
    A2_vals,
    Q_A2_deldop.(A2_vals),
    leg=false,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4)
)
xlabel!("\$A_2 \\times 10^{14} \$ [au/d\$^2]\$")
ylabel!("\$Q(A_2)\$")
#title!("Mean square residual vs A2")
ylims!(-1, 11)
xlims!(-5.9, -5.5)
#savefig("Q_vs_A2.pdf")